In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import sys
sys.path.insert(0, './simglucose')  # 가장 앞에 삽입하여 우선순위 부여

df = pd.read_csv("adolescent#002_100_500.csv")
df["Time"] = pd.to_datetime(df["Time"])

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["Time"], y=df["BG"], name="실제 혈당", line=dict(color="blue")))
fig.add_trace(go.Scatter(x=df["Time"], y=df["CGM"], name="CGM", line=dict(color="green")))
fig.add_trace(go.Scatter(x=df["Time"], y=df["insulin"], name="인슐린 주입량", yaxis="y2", line=dict(color="red")))

fig.update_layout(
    title="혈당 & 인슐린 추이",
    xaxis_title="시간",
    yaxis=dict(title="혈당 (mg/dL)"),
    yaxis2=dict(title="인슐린 (U)", overlaying="y", side="right")
)

st.plotly_chart(fig, use_container_width=True)


2025-06-11 16:48:55.903 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-11 16:48:55.903 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-11 16:48:55.904 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-11 16:48:55.996 
  command:

    streamlit run /home/gyeom/anaconda3/envs/MVP/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-11 16:48:55.997 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [10]:
# 사용자가 목표로 하는 기능을 위한 준비 작업: 특정 시점의 혈당, 인슐린, 식사 정보 추출
# 예시로 중간 시점 (index = 250)을 기준으로 이전 30분(10개 샘플) 동안의 정보 제공

# 타임 포인트 설정
reference_index = 250
context_window = 10  # 약 30분 (3분 간격 * 10)

# 이전 시점 정보 추출
context_df = df.iloc[reference_index - context_window: reference_index + 1][
    ["Time", "BG", "CGM", "CHO", "insulin"]
]

# AI가 추천한 인슐린 주입량: 이후 시점의 insulin 값
ai_insulin = df.iloc[reference_index + 1]["insulin"]

# 이후 혈당 변화 시계열 (AI 주입 기준)
ai_response_df = df.iloc[reference_index + 1 : reference_index + 31][["Time", "BG", "CGM"]]

# 결과 요약
context_df.tail(1), ai_insulin, ai_response_df.head(5)


(                   Time          BG         CGM  CHO   insulin
 250 2025-01-01 12:30:00  159.204321  166.814767  0.0  0.049725,
 0.0769833333333333,
                    Time          BG         CGM
 251 2025-01-01 12:33:00  170.546782  179.143368
 252 2025-01-01 12:36:00  181.483486  190.091604
 253 2025-01-01 12:39:00  191.726158  199.467649
 254 2025-01-01 12:42:00  201.108748  207.204159
 255 2025-01-01 12:45:00  209.565176  213.341857)

In [ ]:
import sys
sys.path.append('./simglucose')  # simglucose 폴더가 있는 경로

from simglucose.simulation.env import T1DSimEnv
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump


from simglucose.controller.base import Action
import simglucose.simulation.env as env
import datetime
import matplotlib.pyplot as plt

# 시뮬레이션 구성 요소 설정
now = datetime.datetime.now()
sensor = CGMSensor.withName("Dexcom")
pump = InsulinPump.withName("Insulet")
patient = T1DPatient.withName("adolescent#002")
test_seed = 42
scenario = RandomScenario(start_time=now, seed=test_seed)

# 환경 초기화
env = T1DSimEnv(patient=patient, sensor=sensor, pump=pump, scenario=scenario)
state = env.reset()

# 사용자 정의 인슐린 주입량 (예: 1.0U)
custom_basal = 0.5

# 시뮬레이션 실행 (90분, 약 30 step)
bg_list = []
cgm_list = []
time_list = []
insulin_list = []
insulin_doses = [0.5, 0.4, 0.3, 0.1, 0.3]

for i in range(len(insulin_doses)):
    obs, reward, done, info = env.step(Action(basal=custom_basal, bolus=insulin_doses[i]))
    bg_list.append(obs[0])
    cgm_list.append(obs[0])
    insulin_list.append(custom_basal)
    # time_list.append(obs.time)

# 시각화를 위해 데이터프레임 구성
sim_result_df = pd.DataFrame({
    # "Time": time_list,
    "BG": bg_list,
    "CGM": cgm_list,
    "Insulin": insulin_list
})

%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(sim_result_df["BG"], label="BG")
plt.plot(sim_result_df["CGM"], label="CGM")
plt.plot(sim_result_df["Insulin"], label="Insulin")

plt.axhline(70, color='red', linestyle='--', linewidth=1)
plt.axhline(180, color='red', linestyle='--', linewidth=1)

plt.title("Simulated Blood Glucose Response")
plt.xlabel("Time Step")
plt.ylabel("mg/dL or Units")
plt.legend()
plt.grid(True)
plt.show()
